In [1]:
# Setup und Konfiguration
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()

ALCHEMY_API_KEY = os.getenv("ALCHEMY_API_KEY")
ALCHEMY_URL = f"https://opt-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"

print("✅ Setup complete")

✅ Setup complete


In [2]:
# === LISTE ALLER VERDÄCHTIGEN BOT-WALLETS ===
# Aus der Forensik-Analyse: Von 0x5babe600 finanzierte Wallets

BOT_WALLETS = [
    # Bestätigter Attacker
    "0x8B6B008A0073D34D04ff00210E7200Ab00003300",
    
    # Top finanzierte Wallets (verdächtig)
    "0x1ee0554c8152bb3795b8d6834b3e458bbcdaf9b7",
    "0xeb256fe71ed492dc4d03ea0b9152d3997bb83c58",
    "0x1712193e45060de2ee7ae1562fe46f826c4428ce",
    "0x452dc5fdc5f1844b254d2c88d5a335b8baa5582d",
    "0x7714f048749cfe8a8bfa86f9730c77563ad7dd31",
]

# Bekannte Profit-Empfänger
KNOWN_RECIPIENTS = {
    "0xda60f6f99cea7fb4acfb3dfcb817691eaa41ba4e": "Recipient 1 (Vanity 0xda60f6)",
    "0xda60192af08b0564fd0f0aab19072a9d4149ba4e": "Recipient 2 (Vanity 0xda60192)",
}

# USDC Contract auf Optimism
USDC_CONTRACT = "0x0b2c639c533813f4aa9d7837caf62653d097ff85"

print(f"📋 Analysiere {len(BOT_WALLETS)} Bot-Wallets")
print(f"🎯 Bekannte Empfänger: {len(KNOWN_RECIPIENTS)}")

📋 Analysiere 6 Bot-Wallets
🎯 Bekannte Empfänger: 2


In [3]:
# === HELPER FUNKTIONEN ===

def get_token_transfers(address: str, direction: str = "from", max_count: int = 500) -> list:
    """
    Hole Token-Transfers für eine Adresse.
    direction: 'from' = ausgehend, 'to' = eingehend
    """
    params = {
        "category": ["erc20"],
        "withMetadata": True,
        "maxCount": hex(max_count)
    }
    
    if direction == "from":
        params["fromAddress"] = address
    else:
        params["toAddress"] = address
    
    response = requests.post(ALCHEMY_URL, json={
        "jsonrpc": "2.0",
        "method": "alchemy_getAssetTransfers",
        "params": [params],
        "id": 1
    })
    
    if response.status_code == 200:
        return response.json().get("result", {}).get("transfers", [])
    return []

print("✅ Helper Funktionen geladen")

✅ Helper Funktionen geladen


In [4]:
# === ANALYSE: USDC Auszahlungen pro Bot ===

print("💰 USDC Auszahlungen der Bot-Wallets")
print("=" * 70)

all_usdc_transfers = []

for bot in BOT_WALLETS:
    print(f"\n📍 Analysiere {bot[:20]}...")
    
    # Hole ausgehende Token-Transfers
    transfers = get_token_transfers(bot, direction="from")
    
    # Filtere nach USDC
    usdc_transfers = [
        tx for tx in transfers 
        if tx.get("rawContract", {}).get("address", "").lower() == USDC_CONTRACT.lower()
        or tx.get("asset", "").upper() == "USDC"
    ]
    
    if usdc_transfers:
        total_usdc = sum(float(tx.get("value", 0) or 0) for tx in usdc_transfers)
        print(f"   ✅ {len(usdc_transfers)} USDC Transfers, Total: {total_usdc:.2f} USDC")
        
        for tx in usdc_transfers:
            recipient = tx.get("to", "").lower()
            recipient_name = KNOWN_RECIPIENTS.get(recipient, "Unknown")
            
            all_usdc_transfers.append({
                "bot": bot,
                "recipient": recipient,
                "recipient_name": recipient_name,
                "amount": float(tx.get("value", 0) or 0),
                "tx_hash": tx.get("hash", ""),
                "timestamp": tx.get("metadata", {}).get("blockTimestamp", "")
            })
    else:
        print(f"   ❌ Keine USDC Transfers gefunden")

print(f"\n\n📊 Gesamt: {len(all_usdc_transfers)} USDC Transfers gefunden")

💰 USDC Auszahlungen der Bot-Wallets

📍 Analysiere 0x8B6B008A0073D34D04...
   ✅ 1 USDC Transfers, Total: 19.83 USDC

📍 Analysiere 0x1ee0554c8152bb3795...
   ❌ Keine USDC Transfers gefunden

📍 Analysiere 0xeb256fe71ed492dc4d...
   ✅ 102 USDC Transfers, Total: 2002.76 USDC

📍 Analysiere 0x1712193e45060de2ee...
   ✅ 14 USDC Transfers, Total: 226.17 USDC

📍 Analysiere 0x452dc5fdc5f1844b25...
   ✅ 28 USDC Transfers, Total: 2580.56 USDC

📍 Analysiere 0x7714f048749cfe8a8b...
   ✅ 160 USDC Transfers, Total: 95849.70 USDC


📊 Gesamt: 305 USDC Transfers gefunden


In [5]:
# === ZUSAMMENFASSUNG ===

if all_usdc_transfers:
    df = pd.DataFrame(all_usdc_transfers)
    
    print("💰 USDC AUSZAHLUNGEN - ZUSAMMENFASSUNG")
    print("=" * 70)
    
    # Gesamt pro Bot
    print("\n📊 USDC pro Bot-Wallet:")
    bot_summary = df.groupby("bot").agg({
        "amount": "sum",
        "tx_hash": "count"
    }).rename(columns={"tx_hash": "transfers"})
    bot_summary = bot_summary.sort_values("amount", ascending=False)
    display(bot_summary)
    
    # Gesamt pro Empfänger
    print("\n🎯 USDC pro Empfänger:")
    recipient_summary = df.groupby(["recipient", "recipient_name"]).agg({
        "amount": "sum",
        "tx_hash": "count"
    }).rename(columns={"tx_hash": "transfers"})
    recipient_summary = recipient_summary.sort_values("amount", ascending=False)
    display(recipient_summary)
    
    # Gesamt
    total_usdc = df["amount"].sum()
    print(f"\n{'='*70}")
    print(f"💵 GESAMT EXTRAHIERT: {total_usdc:.2f} USDC")
    print(f"{'='*70}")
    
else:
    print("❌ Keine USDC Transfers gefunden")

💰 USDC AUSZAHLUNGEN - ZUSAMMENFASSUNG

📊 USDC pro Bot-Wallet:


,amount,transfers
bot,,
0x7714f048749cfe8a8bfa86f9730c77563ad7dd31,95849.696923,160
0x452dc5fdc5f1844b254d2c88d5a335b8baa5582d,2580.557475,28
0xeb256fe71ed492dc4d03ea0b9152d3997bb83c58,2002.759958,102
0x1712193e45060de2ee7ae1562fe46f826c4428ce,226.173676,14
0x8B6B008A0073D34D04ff00210E7200Ab00003300,19.830403,1



🎯 USDC pro Empfänger:


,,amount,transfers
recipient,recipient_name,,
0xef4fb24ad0916217251f553c0596f8edc630eb66,Unknown,27313.768724,9
0x2ed1bd7f66e47113672f3870308b5e867c5bb743,Unknown,9105.281684,9
0xd4a2269719276313aa6ab7b01f91d238eba96433,Unknown,5159.190475,2
0xc7d3ab410d49b664d03fe5b1038852ac852b1b29,Unknown,5064.624795,1
0xf762c3fc745948ff49a3da00ccdc6b755e44305e,Unknown,5008.108641,1
...,...,...,...
0x85b16a35d310db338d7ba35b85f83ea44182a396,Unknown,0.230000,1
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,Unknown,0.120317,1
0xa195608c2306a26f727d5199d5a382a4508308da,Unknown,0.100000,1



💵 GESAMT EXTRAHIERT: 100679.02 USDC


In [6]:
# === DETAILLIERTE TRANSFER-LISTE ===

if all_usdc_transfers:
    print("📋 Alle USDC Transfers (Detail):")
    print("=" * 70)
    
    df_display = df[["bot", "recipient_name", "amount", "timestamp"]].copy()
    df_display["bot"] = df_display["bot"].apply(lambda x: x[:15] + "...")
    df_display["timestamp"] = df_display["timestamp"].apply(lambda x: x[:19] if x else "")
    df_display = df_display.sort_values("timestamp", ascending=False)
    
    display(df_display)

📋 Alle USDC Transfers (Detail):


,bot,recipient_name,amount,timestamp
0,0x8B6B008A0073D...,Recipient 1 (Vanity 0xda60f6),19.830403,2025-11-10T14:14:43
304,0x7714f048749cf...,Unknown,4999.884713,2025-05-07T13:10:33
303,0x7714f048749cf...,Unknown,5000.115287,2025-05-07T11:20:23
302,0x7714f048749cf...,Unknown,5064.624795,2025-05-04T20:06:49
301,0x7714f048749cf...,Unknown,5000.000000,2025-05-04T20:03:01
...,...,...,...,...
149,0x7714f048749cf...,Unknown,58.486740,2022-07-07T13:21:17
148,0x7714f048749cf...,Unknown,750.000000,2022-07-07T13:07:01
147,0x7714f048749cf...,Unknown,42.099775,2022-07-07T12:55:36
146,0x7714f048749cf...,Unknown,57.950000,2022-07-07T12:45:48


In [7]:
# === GEMEINSAMKEITEN DER EMPFÄNGER ANALYSIEREN ===

print("🔍 ANALYSE: Gemeinsamkeiten der Empfänger")
print("=" * 70)

# 1. Empfänger nach Anzahl der erhaltenen Transfers
recipient_counts = df.groupby("recipient").agg({
    "amount": "sum",
    "bot": lambda x: list(set(x)),  # Welche Bots haben an diesen Empfänger gezahlt?
    "tx_hash": "count"
}).rename(columns={"tx_hash": "transfers"})

recipient_counts["num_bots"] = recipient_counts["bot"].apply(len)
recipient_counts = recipient_counts.sort_values("amount", ascending=False)

# 2. Empfänger die von MEHREREN Bots Geld erhalten haben
multi_bot_recipients = recipient_counts[recipient_counts["num_bots"] > 1]

print(f"\n📊 Statistik:")
print(f"   - Gesamtzahl Empfänger: {len(recipient_counts)}")
print(f"   - Empfänger von nur 1 Bot: {len(recipient_counts[recipient_counts['num_bots'] == 1])}")
print(f"   - Empfänger von MEHREREN Bots: {len(multi_bot_recipients)}")

if len(multi_bot_recipients) > 0:
    print(f"\n🎯 VERDÄCHTIG: Empfänger die von mehreren Bots Geld erhalten:")
    print("-" * 70)
    for recipient, row in multi_bot_recipients.iterrows():
        print(f"\n   📍 {recipient}")
        print(f"      💰 Total: {row['amount']:.2f} USDC")
        print(f"      🤖 Anzahl Bots: {row['num_bots']}")
        print(f"      📤 Transfers: {row['transfers']}")
        print(f"      Von: {[b[:15]+'...' for b in row['bot']]}")
else:
    print("\n⚠️ Keine Empfänger gefunden, die von mehreren Bots Geld erhalten haben")

🔍 ANALYSE: Gemeinsamkeiten der Empfänger

📊 Statistik:
   - Gesamtzahl Empfänger: 103
   - Empfänger von nur 1 Bot: 91
   - Empfänger von MEHREREN Bots: 12

🎯 VERDÄCHTIG: Empfänger die von mehreren Bots Geld erhalten:
----------------------------------------------------------------------

   📍 0x2ed1bd7f66e47113672f3870308b5e867c5bb743
      💰 Total: 9105.28 USDC
      🤖 Anzahl Bots: 2
      📤 Transfers: 9
      Von: ['0xeb256fe71ed49...', '0x7714f048749cf...']

   📍 0x000010036c0190e009a000d0fc3541100a07380a
      💰 Total: 5001.15 USDC
      🤖 Anzahl Bots: 2
      📤 Transfers: 2
      Von: ['0xeb256fe71ed49...', '0x7714f048749cf...']

   📍 0xdecc0c09c3b5f6e92ef4184125d5648a66e35298
      💰 Total: 4991.23 USDC
      🤖 Anzahl Bots: 3
      📤 Transfers: 37
      Von: ['0x1712193e45060...', '0x7714f048749cf...', '0x452dc5fdc5f18...']

   📍 0x15392211222b46a0ea85a9a800830486d144848d
      💰 Total: 2124.25 USDC
      🤖 Anzahl Bots: 2
      📤 Transfers: 17
      Von: ['0xeb256fe71ed49...', '